In [3]:
import os
import argparse

import cv2

import rosbag
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
    
def extract_images(bag_file,image_topic,ouput_dir):
    bag = rosbag.Bag(bag_file, "r")
    bridge = CvBridge()
    count = 0
    for topic, msg, t in bag.read_messages(topics=[image_topic]):
        cv_img = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")

        cv_img = cv2.cvtColor(cv_img, cv2.COLOR_RGB2BGR)
        cv_img = cv2.rotate(cv_img, cv2.ROTATE_180)
        cv2.imwrite(os.path.join(output_dir, "frame_%06i.png" % count), cv_img)
        print ("Wrote image %i" % count)

        count += 1

    bag.close()

In [12]:
from pathlib import Path


bag_name = "2021-12-01-13-00-45"

bag = f"../husky_bags/{bag_name}.bag"

!rosbag info {bag}

topic = "/husky/wrist_realsense/color/image_raw"
output_dir = f"./dataset/{bag_name}/"

Path(output_dir).mkdir(parents=True, exist_ok=True)


extract_images(bag,topic,output_dir)

path:        ../husky_bags/2021-12-01-13-00-45.bag
version:     2.0
duration:    1:59s (119s)
start:       Dec 01 2021 13:00:46.32 (1638360046.32)
end:         Dec 01 2021 13:02:45.94 (1638360165.94)
size:        9.2 GB
messages:    3572
compression: none [3572/3572 chunks]
types:       sensor_msgs/Image [060021388200f6f0f447d0fcd9c64743]
topics:      /husky/wrist_realsense/color/image_raw   3572 msgs    : sensor_msgs/Image
Wrote image 0
Wrote image 1
Wrote image 2
Wrote image 3
Wrote image 4
Wrote image 5
Wrote image 6
Wrote image 7
Wrote image 8
Wrote image 9
Wrote image 10
Wrote image 11
Wrote image 12
Wrote image 13
Wrote image 14
Wrote image 15
Wrote image 16
Wrote image 17
Wrote image 18
Wrote image 19
Wrote image 20
Wrote image 21
Wrote image 22
Wrote image 23
Wrote image 24
Wrote image 25
Wrote image 26
Wrote image 27
Wrote image 28
Wrote image 29
Wrote image 30
Wrote image 31
Wrote image 32
Wrote image 33
Wrote image 34
Wrote image 35
Wrote image 36
Wrote image 37
Wrote image 

# Dataset prepare

In [12]:
## remove empty images
## randomize filename

import json
import os
import random, string
from shutil import copyfile

def random_id():
  return ''.join(random.choices(string.ascii_letters + string.digits, k=16))

def init_final_dataset():
  import shutil
  
  shutil.rmtree('./annotated_datasets/final')
  os.mkdir('./annotated_datasets/final')
  os.mkdir('./annotated_datasets/final/images')
  os.mkdir('./annotated_datasets/final/annotations')

  initial_data = {"licenses": [{"name": "", "id": 0, "url": ""}], "info": {"contributor": "", "date_created": "", "description": "", "url": "", "version": "", "year": ""}, "categories": [{"id": 1, "name": "anymal", "supercategory": ""}, {"id": 2, "name": "spot", "supercategory": ""}],"images":[],"annotations":[]}
  
  with open('./annotated_datasets/final/annotations/annotations.json', 'w') as json_file:
    json.dump(initial_data, json_file)




def merge_to_final(dataset_path):
  output_dataset = "./annotated_datasets/final"
  output_images_path = output_dataset+"/images"


  #dataset_path = "annotated_datasets/rosbag-02"
  images_path = dataset_path+"/images/"
  with open(dataset_path+'/annotations/instances_default.json') as f:
    data = json.load(f)

  annotations = data["annotations"]
  images = data["images"]


  with open(output_dataset+'/annotations/annotations.json') as fo:
    output_data = json.load(fo)

  used_image_ids = []
  used_images = []

  annotation_counter = 0
  image_counter = 0

  for image in output_data["images"]:
    if image_counter < image["id"]:
      image_counter = image["id"]

  for annotation in output_data["annotations"]:
    if annotation_counter < annotation["id"]:
      annotation_counter = annotation["id"]

  image_counter += 1
  annotation_counter += 1

  used_annotations = []

  for annotation in annotations:
      used_image_ids.append(annotation['image_id'])
      _annotation = annotation
      _annotation["id"] = _annotation["id"] + annotation_counter
      _annotation["image_id"] = _annotation["image_id"] + image_counter
      used_annotations.append(_annotation)

  for image in images:
      if image["id"] in used_image_ids:
          _image = image
          id = random_id()+".png"
          copyfile(images_path+"/"+image["file_name"], output_images_path+"/"+id)
          _image["file_name"] = id
          _image["id"] = _image["id"] + image_counter
          used_images.append(_image)

  output_data["images"].extend(used_images)
  output_data["annotations"].extend(used_annotations)

  with open(output_dataset+'/annotations/annotations.json', 'w') as json_file:
    json.dump(output_data, json_file)



init_final_dataset()

datasets = [
    "rosbag-01",
    "rosbag-02"
]

for dataset in datasets:
    merge_to_final(f"./annotated_datasets/{dataset}")

## Split Dataset

In [13]:
import json
import random
from shutil import copyfile
from pathlib import Path


annotation_folder = "annotated_datasets/final/annotations/annotations.json"


with open(annotation_folder) as f:
    data = json.load(f)


def createSubDataset(data,output_path,images_subset):

    annotation_output = output_path+"/annotations"
    images_output = output_path+"/images"

    Path(output_path).mkdir(parents=True, exist_ok=True)
    Path(annotation_output).mkdir(parents=True, exist_ok=True)
    Path(images_output).mkdir(parents=True, exist_ok=True)

    #images = data["images"]
    annotations = []

    image_ids = list(map(lambda x: x["id"],images_subset))

    print(image_ids)
    
    for annotation in data["annotations"]:
        if annotation["image_id"] in image_ids:
            annotations.append(annotation)


    sub_data = {
        "licenses": data["licenses"],
        "categories": data["categories"],
        "images": images_subset,
        "annotations":annotations
    }

    with open(annotation_output+"/annotations.json", "w") as file:
        json.dump(sub_data, file)

    for image in images_subset:
        copyfile("annotated_datasets/final/images/"+image["file_name"], images_output+"/"+image["file_name"])


images = data["images"]

random.shuffle(images)

createSubDataset(data,"annotated_datasets/final/train",images[:round(len(images)*0.7)])
createSubDataset(data,"annotated_datasets/final/val",images[round(len(images)*0.7):round(len(images)*0.9)])
createSubDataset(data,"annotated_datasets/final/test",images[round(len(images)*0.9):round(len(images)*1)])

[25, 24, 36, 235, 54, 232, 135, 117, 13, 148, 56, 241, 116, 21, 130, 51, 127, 230, 53, 242, 30, 41, 6, 249, 259, 17, 45, 52, 57, 133, 239]
[142, 3, 38, 115, 125, 256, 132, 244, 223]
[246, 4, 237, 9, 48]


In [2]:
from os import walk
from pathlib import Path


def convertDataset(img_path,ann_path,output_path,colab_path):
  Path(output_path).mkdir(parents=True, exist_ok=True)

  coco_names = "yolo_cfg/custom_coco.names"

  label_output = colab_path+"/images/"
  Path(label_output).mkdir(parents=True, exist_ok=True)

  !python3 ./convert2Yolo/example.py --datasets COCO --img_path {img_path} --label {ann_path} --convert_output_path {label_output} --img_type ".png" --manifest_path ./ --cls_list_file {coco_names}

  filenames = next(walk(label_output), (None, None, []))[2]  # [] if no file

  print(filenames)

  with open(output_path+"/label.txt", "w") as file:
    for filename in filenames:
      #/annotations/yolo/labels/
      file.write(colab_path+"/images/"+filename.replace('.txt', '.png')+"\n")

dataset_path = "annotated_datasets/final"
convertDataset(dataset_path+"/train/images",dataset_path+"/train/annotations/annotations.json",dataset_path+"/train/annotations/yolo",colab_path="spot_anymal_dataset/annotated_datasets/final/train")
convertDataset(dataset_path+"/val/images",dataset_path+"/val/annotations/annotations.json",dataset_path+"/val/annotations/yolo",colab_path="spot_anymal_dataset/annotated_datasets/final/val")










['AQciVlUA7WHUBzu2.txt', 'jXiXmyTljc3lEtef.txt', 'SKUysRG6ZRpndpG1.txt', 'pZgkyXIfQpklaAev.txt', 'SU5xdYJBgMcSkoFH.txt', 'AvSAqs1ZDgHzCp21.txt', 'bpgPHAYWhrGbzooL.txt', '2FLnkModB36W8af0.txt', 'kBjhITAHepYy3qqu.txt', 'gjA8kcxNkIanDpQA.txt', '9NXUyelavxtOFJkS.txt', 'JkRNZ1PsS5HzJx8W.txt', '97PnudavBdRUJivu.txt', 'GBlhGQuK1VuuCpIu.txt', '0OBk3ouCZYHhBHhQ.txt', '8yd9n7RNptqVYyws.txt', 'pNHZGdXIauX2r8y0.txt', '073J8yVYVJs5RZzu.txt', 'ioK94QpBEeijv4Ws.txt', 'wXaOCRQHn8qIkPMk.txt', 'yst6MHEjcAhjS3u4.txt', 'xzSVRXvvADDzpXeR.txt', 'SafgBqBDwz4Ar251.txt', '1RKO8nb3rSHCrov0.txt', 'sCsdFwSTrMLIIhC9.txt', 'yOZ74k5I0eBR7Cs4.txt', 'wetpXd2BuiXLCx6v.txt', 'SMN0aUgaSYfYbxB0.txt', 'oc5KsK75WiUsQhuL.txt', '0hWnXAAVfTKprb1o.txt', 'ap1SJU3XnxVI5elb.txt']









['KMPDpLQtwaTZgFMc.txt', '77jzMIpKuLlUxrrA.txt', '3ZmNyGdTWJx34KS1.txt', 'TyoZHIApLFfOUEgf.txt', '4OfxH8ka28lsAYT3.txt', 'SL5CPjh9faI371Tm.txt', 'dhLVgkmZdQ0GqAS3.txt', '76ObRvMgslDYIYHB.txt', 'vAjC3lfuNMtNDXJV.txt']


## Data Augmentation Pipeline

In [3]:
!pip install -U albumentations

     |████████████████████████████████| 102 kB 2.6 MB/s eta 0:00:01
  Using cached typing_extensions-4.0.1-py3-none-any.whl (22 kB)
